In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from scipy import stats

In [2]:
# Initialize the master DataFrame
master_df = pd.read_csv("ZIPdf.csv")
master_df.rename(columns={"Unnamed: 0" : "ZIP"}, inplace=True)
master_df["ZIP"] = master_df["ZIP"].astype(str).str.zfill(5)
master_df.set_index("ZIP", inplace=True)
master_df

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Gini,UR,%BachOrMore,STCOUNTYFP,W_Avg Emp Min Share,W_Avg Emp Black Share,W_Avg Emp White Share,W_Avg Emp Asian Share,W_Avg Emp Hisp Share,W_Avg Emp Non-Hisp Share
ZIP,,,,,,,,,,,,,,,,,,,,,
00501,NaN,5.0,49.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['36103'],0.149040,NaN,0.859492,0.083086,0.056364,0.892998
00601,NaN,NaN,NaN,NaN,99.785408,99.779608,0.005800,99.669412,0.000000,0.000000,...,44.90,34.986667,14.562560,"['72001', '72113']",NaN,NaN,NaN,NaN,NaN,NaN
00602,NaN,NaN,NaN,NaN,99.432726,99.406094,0.026633,99.280920,0.026633,0.007990,...,49.26,11.262690,21.817471,"['72003', '72005']",NaN,NaN,NaN,NaN,NaN,NaN
00603,NaN,NaN,NaN,NaN,98.827403,98.666774,0.160630,98.377640,0.098386,0.010039,...,57.02,20.851813,24.267387,"['72005', '72071']",NaN,NaN,NaN,NaN,NaN,NaN
00606,NaN,NaN,NaN,NaN,99.480934,99.460970,0.019964,99.301258,0.039928,0.000000,...,43.73,12.011372,10.361217,"['72093', '72153']",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['53063'],0.061404,NaN,0.852339,0.029630,0.020760,0.864522
99256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['53063'],0.061404,NaN,0.852339,0.029630,0.020760,0.864522
99529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['02020'],0.135079,0.015303,0.751177,0.072248,0.027810,0.826074


In [3]:
excluded_prefixes = ('962',
'963',
'964',
'965',
'966',
'090',
'091',
'092',
'093',
'094',
'095',
'096',
'097',
'098',
'006',
'007',
'008',
'009')

master_df = master_df[~master_df.index.str.startswith(excluded_prefixes)]

In [4]:
# List of file names
base = "C:\\Users\\hthog\\Desktop\\PPP Loan Data\\"
file_df = pd.read_csv(base + "PPP Files.csv")
file_df["Completed"] = 0
file_df

,File,Completed
0,public_150k_plus_230630.csv,0
1,public_up_to_150k_1_230630.csv,0
2,public_up_to_150k_2_230630.csv,0
3,public_up_to_150k_3_230630.csv,0
4,public_up_to_150k_4_230630.csv,0
5,public_up_to_150k_5_230630.csv,0
6,public_up_to_150k_6_230630.csv,0
7,public_up_to_150k_7_230630.csv,0
8,public_up_to_150k_8_230630.csv,0
9,public_up_to_150k_9_230630.csv,0


In [5]:
def assign_value_based_on_date(date):
    if date < pd.Timestamp('2020-04-17'):
        return '_1'
    elif date < pd.Timestamp('2020-08-09'):
        return '_2'
    else:
        return '_3'
traunches = ["_1", "_2", "_3"]

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
master_df["Un_Number of Loans_1"] = 0
master_df["Un_Total Loans Amount_1"] = 0
master_df["Un_Employees_1"] = 0
master_df["Re_Number of Loans_1"] = 0
master_df["Re_Total Loans Amount_1"] = 0
master_df["Re_Employees_1"] = 0
master_df["Wh_Number of Loans_1"] = 0
master_df["Wh_Total Loans Amount_1"] = 0
master_df["Wh_Employees_1"] = 0
master_df["Bl_Number of Loans_1"] = 0
master_df["Bl_Total Loans Amount_1"] = 0
master_df["Bl_Employees_1"] = 0

master_df["Un_Number of Loans_2"] = 0
master_df["Un_Total Loans Amount_2"] = 0
master_df["Un_Employees_2"] = 0
master_df["Re_Number of Loans_2"] = 0
master_df["Re_Total Loans Amount_2"] = 0
master_df["Re_Employees_2"] = 0
master_df["Wh_Number of Loans_2"] = 0
master_df["Wh_Total Loans Amount_2"] = 0
master_df["Wh_Employees_2"] = 0
master_df["Bl_Number of Loans_2"] = 0
master_df["Bl_Total Loans Amount_2"] = 0
master_df["Bl_Employees_2"] = 0

master_df["Un_Number of Loans_3"] = 0
master_df["Un_Total Loans Amount_3"] = 0
master_df["Un_Employees_3"] = 0
master_df["Re_Number of Loans_3"] = 0
master_df["Re_Total Loans Amount_3"] = 0
master_df["Re_Employees_3"] = 0
master_df["Wh_Number of Loans_3"] = 0
master_df["Wh_Total Loans Amount_3"] = 0
master_df["Wh_Employees_3"] = 0
master_df["Bl_Number of Loans_3"] = 0
master_df["Bl_Total Loans Amount_3"] = 0
master_df["Bl_Employees_3"] = 0

In [8]:
for file in file_df["File"]:
    file_name = base + file
    # Check if file exists
    if not os.path.exists(file_name):
        print(f"File {file_name} does not exist. Skipping...")
        continue

    print(f"Processing {file}...")
    
    # Read file row by row since cannot fit in DF
    chunk_size = 640000
    updates = []
    for chunk in pd.read_csv(file_name, chunksize=chunk_size, usecols=['NonProfit', 
                                                                        'BusinessType', 
                                                                        'NAICSCode', 
                                                                        'BorrowerZip', 
                                                                        'DateApproved', 
                                                                        'CurrentApprovalAmount',
                                                                        'Race',
                                                                        'Ethnicity',
                                                                        'JobsReported']):
        # Filter chunk to only needed observations
        #chunk = chunk[chunk["NonProfit"] != "Y"]
        chunk = chunk[~chunk["BusinessType"].isin(["Independent Contractors",
                                                   "Self-Employed Individuals"])]
        chunk = chunk[~chunk["NAICSCode"].astype(str).str.startswith(("482",
                                                                      "491",
                                                                      "525110",
                                                                      "525120",
                                                                      "525190",
                                                                      "525920",
                                                                      "541120",
                                                                      "814",
                                                                      "92"))]
        chunk.dropna(subset=["BorrowerZip"], inplace=True)

        # Edit chunk to include necessary info
        chunk["DateApproved"] = pd.to_datetime(chunk["DateApproved"])
        chunk["Traunch"] = chunk["DateApproved"].apply(assign_value_based_on_date)
        chunk["BorrowerZip"] = chunk["BorrowerZip"].astype(str).str[:5]
            
        # Aggregate Data
        grouped_un = chunk[chunk["Race"] == "Unanswered"].groupby(['BorrowerZip', 'Traunch']).size().reset_index(name='Size')       
        grouped_re = chunk[chunk["Race"] != "Unanswered"].groupby(['BorrowerZip', 'Traunch']).size().reset_index(name='Size')
        grouped_wh = chunk[(chunk["Race"] == "White") & (chunk["Ethnicity"] != "Hispanic or Latino")].groupby(['BorrowerZip', 'Traunch']).size().reset_index(name='Size')
        grouped_bl = chunk[chunk["Race"] == "Black or African American"].groupby(['BorrowerZip', 'Traunch']).size().reset_index(name='Size') 
        
        for name, group_df in [("Un", grouped_un), ("Re", grouped_re), ("Wh", grouped_wh), ("Bl", grouped_bl)]:
            for _, row in group_df.iterrows():
                zip_ = row['BorrowerZip']
                traunch = row['Traunch']
                updates.append({
                    "zip": zip_,
                    "traunch": traunch,
                    "key_count": f"{name}_Number of Loans{traunch}",
                    "count": row['Size'], 
                    "update_type": "count"})

        # Edit chunk for amount
        chunk = chunk[~chunk["NAICSCode"].astype(str).str.startswith(("111", "112"))]
        
        # Aggregate for amount
        a_grouped_un = chunk[chunk["Race"] == "Unanswered"].groupby(['BorrowerZip', 'Traunch']).agg({
                        'CurrentApprovalAmount': 'sum', 'JobsReported':'sum'})
        a_grouped_re = chunk[chunk["Race"] != "Unanswered"].groupby(['BorrowerZip', 'Traunch']).agg({
                        'CurrentApprovalAmount': 'sum', 'JobsReported':'sum'})
        a_grouped_wh = chunk[(chunk["Race"] == "White") & (chunk["Ethnicity"] != "Hispanic or Latino")].groupby(['BorrowerZip', 'Traunch']).agg({
                        'CurrentApprovalAmount': 'sum', 'JobsReported':'sum'})
        a_grouped_bl = chunk[chunk["Race"] == "Black or African American"].groupby(['BorrowerZip', 'Traunch']).agg({
                        'CurrentApprovalAmount': 'sum', 'JobsReported':'sum'})
        
        for name, group_df in [("Un", a_grouped_un), ("Re", a_grouped_re), ("Wh", a_grouped_wh), ("Bl", a_grouped_bl)]:
            for (zip_, traunch), row in group_df.iterrows():
                updates.append({
                    "zip": zip_,
                    "traunch": traunch,
                    "key_amount": f"{name}_Total Loans Amount{traunch}",
                    "key_employees": f"{name}_Employees{traunch}",
                    "amount": row['CurrentApprovalAmount'],
                    "jobs": row['JobsReported'],
                    "update_type": "amount"})
        
    # Update master_df
    for update in updates:
        if update['zip'] in master_df.index:
            if update['update_type'] == "count":
                master_df.loc[update['zip'], update['key_count']] += update['count']
            elif update['update_type'] == "amount":
                master_df.loc[update['zip'], update['key_amount']] += update['amount']
                master_df.loc[update['zip'], update['key_employees']] += update['jobs']

    print(f"Updated master DataFrame with data from {file}.")

Processing public_150k_plus_230630.csv...
Updated master DataFrame with data from public_150k_plus_230630.csv.
Processing public_up_to_150k_1_230630.csv...
Updated master DataFrame with data from public_up_to_150k_1_230630.csv.
Processing public_up_to_150k_2_230630.csv...
Updated master DataFrame with data from public_up_to_150k_2_230630.csv.
Processing public_up_to_150k_3_230630.csv...
Updated master DataFrame with data from public_up_to_150k_3_230630.csv.
Processing public_up_to_150k_4_230630.csv...
Updated master DataFrame with data from public_up_to_150k_4_230630.csv.
Processing public_up_to_150k_5_230630.csv...
Updated master DataFrame with data from public_up_to_150k_5_230630.csv.
Processing public_up_to_150k_6_230630.csv...
Updated master DataFrame with data from public_up_to_150k_6_230630.csv.
Processing public_up_to_150k_7_230630.csv...
Updated master DataFrame with data from public_up_to_150k_7_230630.csv.
Processing public_up_to_150k_8_230630.csv...
Updated master DataFrame 

In [9]:
master_df["Un_Total Loans"] = master_df["Un_Number of Loans_1"].add(master_df["Un_Number of Loans_2"], fill_value = 0).add(master_df["Un_Number of Loans_3"], fill_value = 0)
master_df["Un_Total Loan Amount"] = master_df["Un_Total Loans Amount_1"].add(master_df["Un_Total Loans Amount_2"], fill_value = 0).add(master_df["Un_Total Loans Amount_3"], fill_value = 0)
master_df["Un_Employees"] = master_df["Un_Employees_1"].add(master_df["Un_Employees_2"], fill_value = 0).add(master_df["Un_Employees_3"], fill_value = 0)

master_df["Re_Total Loans"] = master_df["Re_Number of Loans_1"].add(master_df["Re_Number of Loans_2"], fill_value = 0).add(master_df["Re_Number of Loans_3"], fill_value = 0)
master_df["Re_Total Loan Amount"] = master_df["Re_Total Loans Amount_1"].add(master_df["Re_Total Loans Amount_2"], fill_value = 0).add(master_df["Re_Total Loans Amount_3"], fill_value = 0)
master_df["Re_Employees"] = master_df["Re_Employees_1"].add(master_df["Re_Employees_2"], fill_value = 0).add(master_df["Re_Employees_3"], fill_value = 0)

master_df["Wh_Total Loans"] = master_df["Wh_Number of Loans_1"].add(master_df["Wh_Number of Loans_2"], fill_value = 0).add(master_df["Wh_Number of Loans_3"], fill_value = 0)
master_df["Wh_Total Loan Amount"] = master_df["Wh_Total Loans Amount_1"].add(master_df["Wh_Total Loans Amount_2"], fill_value = 0).add(master_df["Wh_Total Loans Amount_3"], fill_value = 0)
master_df["Wh_Employees"] = master_df["Wh_Employees_1"].add(master_df["Wh_Employees_2"], fill_value = 0).add(master_df["Wh_Employees_3"], fill_value = 0)

master_df["Bl_Total Loans"] = master_df["Bl_Number of Loans_1"].add(master_df["Bl_Number of Loans_2"], fill_value = 0).add(master_df["Bl_Number of Loans_3"], fill_value = 0)
master_df["Bl_Total Loan Amount"] = master_df["Bl_Total Loans Amount_1"].add(master_df["Bl_Total Loans Amount_2"], fill_value = 0).add(master_df["Bl_Total Loans Amount_3"], fill_value = 0)
master_df["Bl_Employees"] = master_df["Bl_Employees_1"].add(master_df["Bl_Employees_2"], fill_value = 0).add(master_df["Bl_Employees_3"], fill_value = 0)

In [10]:
master_df[["Un_Number of Loans_1",
           "Un_Total Loans Amount_1",
           "Un_Employees_1",
           "Un_Number of Loans_2",
           "Un_Total Loans Amount_2",
           "Un_Employees_2",
           "Un_Number of Loans_3",
           "Un_Total Loans Amount_3",
           "Un_Employees_3",
           "Un_Total Loans",
           "Un_Total Loan Amount",
           "Un_Employees",
        "Re_Number of Loans_1",
           "Re_Total Loans Amount_1",
           "Re_Employees_1",
           "Re_Number of Loans_2",
           "Re_Total Loans Amount_2",
           "Re_Employees_2",
           "Re_Number of Loans_3",
           "Re_Total Loans Amount_3",
           "Re_Employees_3",
           "Re_Total Loans",
           "Re_Total Loan Amount",
           "Re_Employees",
        "Wh_Number of Loans_1",
           "Wh_Total Loans Amount_1",
           "Wh_Employees_1",
           "Wh_Number of Loans_2",
           "Wh_Total Loans Amount_2",
           "Wh_Employees_2",
           "Wh_Number of Loans_3",
           "Wh_Total Loans Amount_3",
           "Wh_Employees_3",
           "Wh_Total Loans",
           "Wh_Total Loan Amount",
           "Wh__Employees",
       "Bl_Number of Loans_1",
           "Bl_Total Loans Amount_1",
           "Bl_Employees_1",
           "Bl_Number of Loans_2",
           "Bl_Total Loans Amount_2",
           "Bl_Employees_2",
           "Bl_Number of Loans_3",
           "Bl_Total Loans Amount_3",
           "Bl_Employees_3",
           "Bl_Total Loans",
           "Bl_Total Loan Amount",
           "Bl_Employees"]] = master_df[["Un_Number of Loans_1",
                                           "Un_Total Loans Amount_1",
                                           "Un_Employees_1",
                                           "Un_Number of Loans_2",
                                           "Un_Total Loans Amount_2",
                                           "Un_Employees_2",
                                           "Un_Number of Loans_3",
                                           "Un_Total Loans Amount_3",
                                           "Un_Employees_3",
                                           "Un_Total Loans",
                                           "Un_Total Loan Amount",
                                           "Un_Employees",
                                        "Re_Number of Loans_1",
                                           "Re_Total Loans Amount_1",
                                           "Re_Employees_1",
                                           "Re_Number of Loans_2",
                                           "Re_Total Loans Amount_2",
                                           "Re_Employees_2",
                                           "Re_Number of Loans_3",
                                           "Re_Total Loans Amount_3",
                                           "Re_Employees_3",
                                           "Re_Total Loans",
                                           "Re_Total Loan Amount",
                                           "Re_Employees",
                                        "Wh_Number of Loans_1",
                                           "Wh_Total Loans Amount_1",
                                           "Wh_Employees_1",
                                           "Wh_Number of Loans_2",
                                           "Wh_Total Loans Amount_2",
                                           "Wh_Employees_2",
                                           "Wh_Number of Loans_3",
                                           "Wh_Total Loans Amount_3",
                                           "Wh_Employees_3",
                                           "Wh_Total Loans",
                                           "Wh_Total Loan Amount",
                                           "Wh_Employees",
                                       "Bl_Number of Loans_1",
                                           "Bl_Total Loans Amount_1",
                                           "Bl_Employees_1",
                                           "Bl_Number of Loans_2",
                                           "Bl_Total Loans Amount_2",
                                           "Bl_Employees_2",
                                           "Bl_Number of Loans_3",
                                           "Bl_Total Loans Amount_3",
                                           "Bl_Employees_3",
                                           "Bl_Total Loans",
                                           "Bl_Total Loan Amount",
                                           "Bl_Employees"]].fillna(0)

# Dividing by the total might be problematic

In [11]:
master_df["Un_Loans per Emp Bus"] = master_df["Un_Total Loans"] / master_df["Total Emp Bus"]
master_df["Un_Avg Loan Amt per Emp"] = master_df["Un_Total Loan Amount"] / master_df["Un_Employees"]
master_df["Un_Loans per Emp Bus_1"] = master_df["Un_Number of Loans_1"] / master_df["Total Emp Bus"]
master_df["Un_Avg Loan Amt per Emp_1"] = master_df["Un_Total Loans Amount_1"] / master_df["Un_Employees_1"]
master_df["Un_Loans per Emp Bus_2"] = master_df["Un_Number of Loans_2"] / master_df["Total Emp Bus"]
master_df["Un_Avg Loan Amt per Emp_2"] = master_df["Un_Total Loans Amount_2"] / master_df["Un_Employees_2"]
master_df["Un_Loans per Emp Bus_3"] = master_df["Un_Number of Loans_3"] / master_df["Total Emp Bus"]
master_df["Un_Avg Loan Amt per Emp_3"] = master_df["Un_Total Loans Amount_3"] / master_df["Un_Employees_3"]

master_df["Re_Loans per Emp Bus"] = master_df["Re_Total Loans"] / master_df["Total Emp Bus"]
master_df["Re_Avg Loan Amt per Emp"] = master_df["Re_Total Loan Amount"] / master_df["Re_Employees"]
master_df["Re_Loans per Emp Bus_1"] = master_df["Re_Number of Loans_1"] / master_df["Total Emp Bus"]
master_df["Re_Avg Loan Amt per Emp_1"] = master_df["Re_Total Loans Amount_1"] / master_df["Re_Employees_1"]
master_df["Re_Loans per Emp Bus_2"] = master_df["Re_Number of Loans_2"] / master_df["Total Emp Bus"]
master_df["Re_Avg Loan Amt per Emp_2"] = master_df["Re_Total Loans Amount_2"] / master_df["Re_Employees_2"]
master_df["Re_Loans per Emp Bus_3"] = master_df["Re_Number of Loans_3"] / master_df["Total Emp Bus"]
master_df["Re_Avg Loan Amt per Emp_3"] = master_df["Re_Total Loans Amount_3"] / master_df["Re_Employees_3"]

master_df["Wh_Loans per Emp Bus"] = master_df["Wh_Total Loans"] / master_df["Total Emp Bus"]
master_df["Wh_Avg Loan Amt per Emp"] = master_df["Wh_Total Loan Amount"] / master_df["Wh_Employees"]
master_df["Wh_Loans per Emp Bus_1"] = master_df["Wh_Number of Loans_1"] / master_df["Total Emp Bus"]
master_df["Wh_Avg Loan Amt per Emp_1"] = master_df["Wh_Total Loans Amount_1"] / master_df["Wh_Employees_1"]
master_df["Wh_Loans per Emp Bus_2"] = master_df["Wh_Number of Loans_2"] / master_df["Total Emp Bus"]
master_df["Wh_Avg Loan Amt per Emp_2"] = master_df["Wh_Total Loans Amount_2"] / master_df["Wh_Employees_2"]
master_df["Wh_Loans per Emp Bus_3"] = master_df["Wh_Number of Loans_3"] / master_df["Total Emp Bus"]
master_df["Wh_Avg Loan Amt per Emp_3"] = master_df["Wh_Total Loans Amount_3"] / master_df["Wh_Employees_3"]

master_df["Bl_Loans per Emp Bus"] = master_df["Bl_Total Loans"] / master_df["Total Emp Bus"]
master_df["Bl_Avg Loan Amt per Emp"] = master_df["Bl_Total Loan Amount"] / master_df["Bl_Employees"]
master_df["Bl_Loans per Emp Bus_1"] = master_df["Bl_Number of Loans_1"] / master_df["Total Emp Bus"]
master_df["Bl_Avg Loan Amt per Emp_1"] = master_df["Bl_Total Loans Amount_1"] / master_df["Bl_Employees_1"]
master_df["Bl_Loans per Emp Bus_2"] = master_df["Bl_Number of Loans_2"] / master_df["Total Emp Bus"]
master_df["Bl_Avg Loan Amt per Emp_2"] = master_df["Bl_Total Loans Amount_2"] / master_df["Bl_Employees_2"]
master_df["Bl_Loans per Emp Bus_3"] = master_df["Bl_Number of Loans_3"] / master_df["Total Emp Bus"]
master_df["Bl_Avg Loan Amt per Emp_3"] = master_df["Bl_Total Loans Amount_3"] / master_df["Bl_Employees_3"]
master_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [12]:
from scipy import stats

In [14]:
master_df.to_csv("PPP by Race Reported.csv")

In [15]:
master_df = pd.read_csv('PPP by Race Reported.csv')
master_df.rename(columns={"Unnamed: 0" : "ZIP"}, inplace=True)
master_df["ZIP"] = master_df["ZIP"].astype(str).str.zfill(5)
master_df.set_index("ZIP", inplace=True)
master_df

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Min Share Excl B,Black Share,Hisp Share,Asian Share,Native Share,...,Wh_Loans per Emp Bus_3,Wh_Avg Loan Amt per Emp_3,Bl_Loans per Emp Bus,Bl_Avg Loan Amt per Emp,Bl_Loans per Emp Bus_1,Bl_Avg Loan Amt per Emp_1,Bl_Loans per Emp Bus_2,Bl_Avg Loan Amt per Emp_2,Bl_Loans per Emp Bus_3,Bl_Avg Loan Amt per Emp_3
ZIP,,,,,,,,,,,,,,,,,,,,,
00501,NaN,5.0,49.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN
01001,11.0,469.0,7944.0,480.0,15.390956,13.312529,2.078427,6.771079,2.890956,0.135422,...,0.102083,8849.537303,0.008333,2655.112903,0.002083,2187.693333,0.0,NaN,0.006250,3093.318750
01002,60.0,523.0,7732.0,583.0,36.718920,31.678641,5.040279,9.902751,15.712316,0.166921,...,0.099485,9415.454695,0.005146,17369.416667,0.000000,NaN,0.0,NaN,0.005146,17369.416667
01003,3.0,16.0,239.0,19.0,39.568400,32.943484,6.624915,8.737644,20.176564,0.105636,...,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN
01004,3.0,9.0,197.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.083333,23382.333333,0.083333,3100.000000,0.000000,NaN,0.0,NaN,0.083333,3100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
master_df[["Wh_Avg Loan Amt per Emp_1",
           "Wh_Avg Loan Amt per Emp_2",
           "Wh_Avg Loan Amt per Emp_3",
           "Wh_Avg Loan Amt per Emp"]].describe()

,Wh_Avg Loan Amt per Emp_1,Wh_Avg Loan Amt per Emp_2,Wh_Avg Loan Amt per Emp_3,Wh_Avg Loan Amt per Emp
count,22674.000000,2.384800e+04,26984.000000,29793.000000
mean,8621.339243,7.351200e+03,9091.241139,8403.355677
std,4274.763671,1.214148e+04,4027.770566,3713.563600
min,98.494000,7.600000e+00,140.000000,7.600000
25%,6051.626522,4.809968e+03,6647.657808,6175.000000
50%,8170.428725,6.716633e+03,8549.090630,7951.736000
75%,10482.593919,8.912948e+03,10767.505228,9934.351429
max,103996.296296,1.779400e+06,70400.000000,66213.375000


In [32]:
master_df["Un_Number of Loans"] = master_df["Un_Number of Loans_1"] + master_df["Un_Number of Loans_2"] + master_df["Un_Number of Loans_3"]
master_df["Re_Number of Loans"] = master_df["Re_Number of Loans_1"] + master_df["Re_Number of Loans_2"] + master_df["Re_Number of Loans_3"]
master_df["Bl_Number of Loans"] = master_df["Bl_Number of Loans_1"] + master_df["Bl_Number of Loans_2"] + master_df["Bl_Number of Loans_3"]
master_df["Wh_Number of Loans"] = master_df["Wh_Number of Loans_1"] + master_df["Wh_Number of Loans_2"] + master_df["Wh_Number of Loans_3"]

In [33]:
master_df[["Un_Avg Loan Amt per Emp_1",
           "Un_Avg Loan Amt per Emp_2",
           "Un_Avg Loan Amt per Emp_3",
           "Un_Avg Loan Amt per Emp"]].describe()

,Un_Avg Loan Amt per Emp_1,Un_Avg Loan Amt per Emp_2,Un_Avg Loan Amt per Emp_3,Un_Avg Loan Amt per Emp
count,30507.000000,33111.000000,32084.000000,35931.000000
mean,8790.124949,7118.792572,9003.678527,8301.423456
std,4276.936747,3504.481863,3683.856514,3393.778269
min,227.187500,7.664671,150.000000,96.528600
25%,6593.534851,5170.485413,6914.050017,6502.213153
50%,8503.497316,6690.477273,8570.373870,7990.512335
75%,10549.788290,8409.745132,10415.550650,9627.539680
max,389980.000000,126056.157895,95000.000000,198107.372000


In [34]:
black_zips = master_df[master_df["Black Share"] >= 75]
black_zips[["Wh_Avg Loan Amt per Emp_1",
           "Wh_Avg Loan Amt per Emp_2",
           "Wh_Avg Loan Amt per Emp_3",
           "Wh_Avg Loan Amt per Emp"]].describe()

,Wh_Avg Loan Amt per Emp_1,Wh_Avg Loan Amt per Emp_2,Wh_Avg Loan Amt per Emp_3,Wh_Avg Loan Amt per Emp
count,194.000000,207.000000,240.000000,257.000000
mean,9624.989683,8154.621754,10250.085388,9473.154537
std,4145.015995,4189.001527,4372.861957,3802.844721
min,1235.597500,821.287500,769.000000,821.287500
25%,6907.478334,5213.469643,7292.441865,7080.961923
50%,9355.241665,7877.882353,9239.582522,8939.231665
75%,11766.074020,10440.000000,12504.205000,11354.000000
max,22917.000000,25970.750000,28359.973333,20833.330000


In [35]:
black_zips[["Un_Avg Loan Amt per Emp_1",
           "Un_Avg Loan Amt per Emp_2",
           "Un_Avg Loan Amt per Emp_3",
           "Un_Avg Loan Amt per Emp"]].describe()

,Un_Avg Loan Amt per Emp_1,Un_Avg Loan Amt per Emp_2,Un_Avg Loan Amt per Emp_3,Un_Avg Loan Amt per Emp
count,273.000000,300.000000,316.000000,325.000000
mean,9201.755025,7029.912100,12674.933208,10420.895792
std,2864.250925,4477.240947,5697.837413,5620.966994
min,1058.000000,282.075421,1733.856250,1774.503846
25%,7117.086408,5175.322068,10338.192548,8407.944505
50%,9109.646035,6557.030710,12158.272827,9784.909813
75%,10995.515517,8084.927332,14326.759003,11476.621902
max,19985.937500,64158.941667,95000.000000,95000.000000


In [36]:
white_zips = master_df[master_df["White Share"] >= 75]#[["Un_Avg Loan Amt per Emp_1",
#                                            "Un_Avg Loan Amt per Emp_2",
#                                            "Un_Avg Loan Amt per Emp_3",
#                                            "Un_Avg Loan Amt per Emp"]]
white_zips[["Wh_Avg Loan Amt per Emp_1",
           "Wh_Avg Loan Amt per Emp_2",
           "Wh_Avg Loan Amt per Emp_3",
           "Wh_Avg Loan Amt per Emp"]].describe()

,Wh_Avg Loan Amt per Emp_1,Wh_Avg Loan Amt per Emp_2,Wh_Avg Loan Amt per Emp_3,Wh_Avg Loan Amt per Emp
count,12967.000000,13734.000000,16567.000000,17671.000000
mean,8105.177567,6952.890169,8856.503351,8074.178893
std,4116.114581,4028.872128,4180.103090,3660.026548
min,98.494000,49.356574,140.000000,180.333333
25%,5621.542697,4425.522083,6270.494247,5840.325075
50%,7696.558550,6347.683761,8227.015063,7624.222380
75%,9893.975944,8560.321870,10557.963000,9566.732232
max,100000.000000,74699.000000,66213.375000,66213.375000


In [37]:
white_zips[["Un_Avg Loan Amt per Emp_1",
           "Un_Avg Loan Amt per Emp_2",
           "Un_Avg Loan Amt per Emp_3",
           "Un_Avg Loan Amt per Emp"]].describe()

,Un_Avg Loan Amt per Emp_1,Un_Avg Loan Amt per Emp_2,Un_Avg Loan Amt per Emp_3,Un_Avg Loan Amt per Emp
count,17149.000000,18452.000000,19058.000000,20095.000000
mean,8352.335062,6811.696408,8503.371450,7908.007916
std,4694.965203,3256.276252,3457.710670,3230.043319
min,227.187500,146.037773,200.000000,222.000000
25%,6179.781069,4901.696670,6482.409751,6196.857692
50%,8035.641558,6433.677591,8081.211633,7659.232745
75%,10026.095844,8152.160317,9897.478511,9194.143560
max,389980.000000,71655.000000,60386.911765,198107.372000


In [38]:
min_zips = master_df[master_df["White Share"] <= 75]#[["Un_Avg Loan Amt per Emp_1",
#                                            "Un_Avg Loan Amt per Emp_2",
#                                            "Un_Avg Loan Amt per Emp_3",
#                                            "Un_Avg Loan Amt per Emp"]]
min_zips[["Wh_Avg Loan Amt per Emp_1",
           "Wh_Avg Loan Amt per Emp_2",
           "Wh_Avg Loan Amt per Emp_3",
           "Wh_Avg Loan Amt per Emp"]].describe()

,Wh_Avg Loan Amt per Emp_1,Wh_Avg Loan Amt per Emp_2,Wh_Avg Loan Amt per Emp_3,Wh_Avg Loan Amt per Emp
count,8694.000000,9.117000e+03,9986.000000,10349.000000
mean,9183.953680,7.744442e+03,9391.215743,8712.402685
std,4134.438633,1.889111e+04,3539.197919,3286.447443
min,208.333333,4.578060e+01,207.000000,106.746535
25%,6724.209678,5.396617e+03,7292.216838,6761.551444
50%,8758.172423,7.164433e+03,8994.174991,8372.986250
75%,11053.755000,9.123720e+03,10912.325215,10174.528819
max,103996.296296,1.779400e+06,70400.000000,50633.333333


In [39]:
min_zips[["Un_Avg Loan Amt per Emp_1",
           "Un_Avg Loan Amt per Emp_2",
           "Un_Avg Loan Amt per Emp_3",
           "Un_Avg Loan Amt per Emp"]].describe()

,Un_Avg Loan Amt per Emp_1,Un_Avg Loan Amt per Emp_2,Un_Avg Loan Amt per Emp_3,Un_Avg Loan Amt per Emp
count,10599.000000,11257.000000,11343.000000,11764.000000
mean,9150.537285,7126.670521,9575.150806,8588.270948
std,2984.086764,2836.333478,3251.027653,2796.995867
min,486.089239,7.664671,447.000000,96.528600
25%,7265.944149,5569.558824,7837.292611,7087.487187
50%,9030.339119,6845.073388,9214.757483,8366.679704
75%,10852.733102,8282.292838,10786.851315,9771.804347
max,37500.000000,72200.000000,95000.000000,95000.000000


In [40]:
for df in [black_zips, white_zips, min_zips, master_df]:
    for tranche in ["_1", "_2", "_3", ""]:
        t_stat, p_val = stats.ttest_ind(df[f"Wh_Avg Loan Amt per Emp{tranche}"].dropna(), df[f"Un_Avg Loan Amt per Emp{tranche}"].dropna())
        print(tranche)
        print(df[f"Wh_Avg Loan Amt per Emp{tranche}"].mean() - df[f"Un_Avg Loan Amt per Emp{tranche}"].mean())
        print(t_stat, p_val)

_1
423.2346572278566
1.3049245561296046 0.19256392770510106
_2
1124.7096536341724
2.853647225761435 0.004499519273309628
_3
-2424.847820185647
-5.479845789560343 6.471304633743356e-08

-947.7412552081951
-2.3159758930091723 0.02090736626248123
_1
-247.15749472599873
-4.7672750630513825 1.8760592596536175e-06
_2
141.19376044070668
3.474135238159936 0.0005131766332026174
_3
353.1319011221367
8.72394354828791 2.8010462701666795e-18

166.17097718332116
4.686854218031199 2.784033452463076e-06
_1
33.416394982285055
0.6507368891453711 0.5152240962814982
_2
617.7710373321625
3.422332904732218 0.0006220824401402849
_3
-183.9350631960915
-3.9551946937112765 7.671946608500441e-05

124.13173751710565
3.0338792396149925 0.0024171125791509437
_1
-168.7857061548366
-4.501761536150088 6.753542068860951e-06
_2
232.40759206254734
3.297598818011311 0.0009757504663295305
_3
87.5626114622155
2.757198091833398 0.00583169221770735

101.93222039888315
3.6724326797288427 0.00024044282231112666


In [41]:
import scipy.stats as stats

for df in [black_zips, white_zips, min_zips, master_df]:
    for tranche in ["_1", "_2", "_3", ""]:
        series_master = df[f"Bl_Avg Loan Amt per Emp{tranche}"]
        series_df = df[f"Wh_Avg Loan Amt per Emp{tranche}"]

        # Check if the series are non-empty and have variance
        if series_master.empty or series_df.empty:
            print(f"Empty series for tranche {tranche}")
            continue

        if np.all(series_master == series_master.iloc[0]) or np.all(series_df == series_df.iloc[0]):
            print(f"No variance in data for tranche {tranche}")
            continue

        # Performing the t-test
        t_stat, p_val = stats.ttest_ind(series_master.dropna(), series_df.dropna())

        if np.isnan(t_stat):
            print(f"NaN result for t-test in tranche {tranche}")
        else:
            mean_diff = series_master.mean() - series_df.mean()
            print(tranche)
            print("Mean Difference:", mean_diff)
            print("T-statistic:", t_stat, "P-value:", p_val)

_1
Mean Difference: -3433.2955229882527
T-statistic: -8.032753857928581 P-value: 1.4017833671049157e-14
_2
Mean Difference: -1784.7115866819913
T-statistic: -4.759615715372953 P-value: 2.613280391587139e-06
_3
Mean Difference: 3738.61587778797
T-statistic: 11.51042527210086 P-value: 1.3392657135970892e-27

Mean Difference: 3025.6612952404994
T-statistic: 9.626853624337658 P-value: 2.0740706784327835e-20
_1
Mean Difference: 3674.268434952871
T-statistic: 3.6586130503998735 P-value: 0.0002545236550194212
_2
Mean Difference: 656.3183195864294
T-statistic: 6.1555690375104115 P-value: 7.662801992579934e-10
_3
Mean Difference: 4115.35040841587
T-statistic: 52.887324979974956 P-value: 0.0

Mean Difference: 4507.6472981034
T-statistic: 11.961951573676467 P-value: 7.079098645357639e-33
_1
Mean Difference: -1419.356564575377
T-statistic: -13.780096270342614 P-value: 7.165831936011664e-43
_2
Mean Difference: -546.6016956730864
T-statistic: -2.2514616439169086 P-value: 0.02437025551356283
_3
Mean 

In [51]:
print(master_df["Re_Number of Loans_1"].sum() / (master_df["Un_Number of Loans_1"].sum() + master_df["Re_Number of Loans_1"].sum()))
print(master_df["Re_Number of Loans_2"].sum() / (master_df["Un_Number of Loans_2"].sum() + master_df["Re_Number of Loans_2"].sum()))
print(master_df["Re_Number of Loans_3"].sum() / (master_df["Un_Number of Loans_3"].sum() + master_df["Re_Number of Loans_3"].sum()))
print(master_df["Re_Number of Loans"].sum() / (master_df["Un_Number of Loans"].sum() + master_df["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(master_df["Bl_Number of Loans_1"].sum() / (master_df["Re_Number of Loans_1"].sum()))
print(master_df["Bl_Number of Loans_2"].sum() / (master_df["Re_Number of Loans_2"].sum()))
print(master_df["Bl_Number of Loans_3"].sum() / (master_df["Re_Number of Loans_3"].sum()))
print(master_df["Bl_Number of Loans"].sum() / (master_df["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(master_df["Wh_Number of Loans_1"].sum() / (master_df["Re_Number of Loans_1"].sum()))
print(master_df["Wh_Number of Loans_2"].sum() / (master_df["Re_Number of Loans_2"].sum()))
print(master_df["Wh_Number of Loans_3"].sum() / (master_df["Re_Number of Loans_3"].sum()))
print(master_df["Wh_Number of Loans"].sum() / (master_df["Re_Number of Loans"].sum()))

0.20839691391493265
0.17007571487737813
0.3101933796327426
0.2476993429678199
-------------------------------------
0.02409460369009927
0.1044799460835147
0.3781789533038586
0.26783175941584775
-------------------------------------
0.8222257099653204
0.5988118266257328
0.4416201635016572
0.5295109006709936


In [52]:
print(white_zips["Re_Number of Loans_1"].sum() / (white_zips["Un_Number of Loans_1"].sum() + white_zips["Re_Number of Loans_1"].sum()))
print(white_zips["Re_Number of Loans_2"].sum() / (white_zips["Un_Number of Loans_2"].sum() + white_zips["Re_Number of Loans_2"].sum()))
print(white_zips["Re_Number of Loans_3"].sum() / (white_zips["Un_Number of Loans_3"].sum() + white_zips["Re_Number of Loans_3"].sum()))
print(white_zips["Re_Number of Loans"].sum() / (white_zips["Un_Number of Loans"].sum() + white_zips["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(white_zips["Bl_Number of Loans_1"].sum() / (white_zips["Re_Number of Loans_1"].sum()))
print(white_zips["Bl_Number of Loans_2"].sum() / (white_zips["Re_Number of Loans_2"].sum()))
print(white_zips["Bl_Number of Loans_3"].sum() / (white_zips["Re_Number of Loans_3"].sum()))
print(white_zips["Bl_Number of Loans"].sum() / (white_zips["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(white_zips["Wh_Number of Loans_1"].sum() / (white_zips["Re_Number of Loans_1"].sum()))
print(white_zips["Wh_Number of Loans_2"].sum() / (white_zips["Re_Number of Loans_2"].sum()))
print(white_zips["Wh_Number of Loans_3"].sum() / (white_zips["Re_Number of Loans_3"].sum()))
print(white_zips["Wh_Number of Loans"].sum() / (white_zips["Re_Number of Loans"].sum()))

0.22433180997824548
0.17088136434388324
0.27650802168610045
0.2321196071304251
-------------------------------------
0.008346774722320574
0.02519051905190519
0.06939897186708345
0.046367565626855574
-------------------------------------
0.9253970127332573
0.8376837683768377
0.8364567438507083
0.855470400596545


In [53]:
print(black_zips["Re_Number of Loans_1"].sum() / (black_zips["Un_Number of Loans_1"].sum() + black_zips["Re_Number of Loans_1"].sum()))
print(black_zips["Re_Number of Loans_2"].sum() / (black_zips["Un_Number of Loans_2"].sum() + black_zips["Re_Number of Loans_2"].sum()))
print(black_zips["Re_Number of Loans_3"].sum() / (black_zips["Un_Number of Loans_3"].sum() + black_zips["Re_Number of Loans_3"].sum()))
print(black_zips["Re_Number of Loans"].sum() / (black_zips["Un_Number of Loans"].sum() + black_zips["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(black_zips["Bl_Number of Loans_1"].sum() / (black_zips["Re_Number of Loans_1"].sum()))
print(black_zips["Bl_Number of Loans_2"].sum() / (black_zips["Re_Number of Loans_2"].sum()))
print(black_zips["Bl_Number of Loans_3"].sum() / (black_zips["Re_Number of Loans_3"].sum()))
print(black_zips["Bl_Number of Loans"].sum() / (black_zips["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(black_zips["Wh_Number of Loans_1"].sum() / (black_zips["Re_Number of Loans_1"].sum()))
print(black_zips["Wh_Number of Loans_2"].sum() / (black_zips["Re_Number of Loans_2"].sum()))
print(black_zips["Wh_Number of Loans_3"].sum() / (black_zips["Re_Number of Loans_3"].sum()))
print(black_zips["Wh_Number of Loans"].sum() / (black_zips["Re_Number of Loans"].sum()))

0.23995844875346262
0.22266015029691524
0.42387121425777163
0.3891656662665066
-------------------------------------
0.2356902356902357
0.6899929195185273
0.9385724863977335
0.9041798411351893
-------------------------------------
0.543049543049543
0.11989615293839981
0.02714014358842278
0.0450566823474975


In [54]:
print(min_zips["Re_Number of Loans_1"].sum() / (min_zips["Un_Number of Loans_1"].sum() + min_zips["Re_Number of Loans_1"].sum()))
print(min_zips["Re_Number of Loans_2"].sum() / (min_zips["Un_Number of Loans_2"].sum() + min_zips["Re_Number of Loans_2"].sum()))
print(min_zips["Re_Number of Loans_3"].sum() / (min_zips["Un_Number of Loans_3"].sum() + min_zips["Re_Number of Loans_3"].sum()))
print(min_zips["Re_Number of Loans"].sum() / (min_zips["Un_Number of Loans"].sum() + min_zips["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(min_zips["Bl_Number of Loans_1"].sum() / (min_zips["Re_Number of Loans_1"].sum()))
print(min_zips["Bl_Number of Loans_2"].sum() / (min_zips["Re_Number of Loans_2"].sum()))
print(min_zips["Bl_Number of Loans_3"].sum() / (min_zips["Re_Number of Loans_3"].sum()))
print(min_zips["Bl_Number of Loans"].sum() / (min_zips["Re_Number of Loans"].sum()))
print("-------------------------------------")
print(min_zips["Wh_Number of Loans_1"].sum() / (min_zips["Re_Number of Loans_1"].sum()))
print(min_zips["Wh_Number of Loans_2"].sum() / (min_zips["Re_Number of Loans_2"].sum()))
print(min_zips["Wh_Number of Loans_3"].sum() / (min_zips["Re_Number of Loans_3"].sum()))
print(min_zips["Wh_Number of Loans"].sum() / (min_zips["Re_Number of Loans"].sum()))

0.19684547017006507
0.16977332392560635
0.3244753210455999
0.25544777837464505
-------------------------------------
0.03768253806474262
0.14052965335783268
0.49014878618972696
0.36485577125278634
-------------------------------------
0.7327965482681887
0.49061024569109624
0.29893861922272563
0.3873435594248875
